# Neural style transfer

This notebook is based on the first portion of the 2018 TensorFlow Authors [Neural Style Transfer](https://www.tensorflow.org/tutorials/generative/style_transfer) tutorial, the notebook from which was released under the [Apache License, version 2.0](https://www.apache.org/licenses/LICENSE-2.0). I have made a few modifications, mostly related to data loading from a local filepath vs. a web url.  You do not need a GPU or TPU to get good results fast.

### Import and configure modules

In [ ]:
import tensorflow as tf
from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image

In [ ]:
def tensor_to_image(tensor):

    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

### Choose a content image and a style image (1st Photo and 1st Monet):

In [ ]:
content_path_orig = '../input/gan-getting-started/photo_jpg/00068bc07f.jpg'
style_path_orig = '../input/gan-getting-started/monet_jpg/000c1e3bff.jpg'

## Visualize the input

### Define a function to load an image and limit its maximum dimension to 256 pixels.

In [ ]:
def load_img(path_to_img):
    max_dim = 256
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

### Create a simple function to display an image:

In [ ]:
def imshow(image, title=None):
    # convert tensor elements to 0 - 255 values
    image = image * 255
    
    # Remove batch dimension of tensor
    if len(image.shape) > 3:
        out = np.squeeze(image, axis=0)
     
    out = out.astype('uint8')
    plt.imshow(out)
    if title:
        plt.title(title)
    plt.imshow(out)

In [ ]:
plt.figure(figsize=(10,10))

content_image_orig = load_img(content_path_orig)
style_image_orig = load_img(style_path_orig)

plt.subplot(1, 2, 1)
imshow(content_image_orig, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image_orig, 'Style Image')

## Fast Style Transfer using TF-Hub

### Let's download the [TensorFlow Hub style transfer model](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) -- just 81.66 MB.
### There are versions for mobile devices too!

In [ ]:
import tensorflow_hub as hub
# This is version is updated from the original notebook (/2 vs. /1)
# Results are greatly improved
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

### Run the model and display result

In [ ]:
stylized_image_orig = hub_module(tf.constant(content_image_orig), tf.constant(style_image_orig))[0]
plt.figure(figsize=(15,15))

plt.subplot(1, 3, 1)
imshow(content_image_orig, 'Content Image')

plt.subplot(1, 3, 2)
imshow(stylized_image_orig, 'Result')

plt.subplot(1, 3, 3)
imshow(style_image_orig, 'Style Image')

## Is it a Monet?

If Claude Monet had set up his easel in the field of the content image, would he have painted something like the result image?  Although the color palette looks pretty close to the style image, the style itself seems off.  In Monet's painting (which might be *At Val-Saint-Nicholas near Dieppe (Morning)*, 1897 from the Phillips Collection in Washington, DC), the sea, sky, and cliffs are indistinct. Yet, in the result image, the mountains, clouds, and field are fairly detailed. Have we gotten it wrong?

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Monet_-_Valle_Buona%2C_Near_Bordighera%2C_1884.jpg/640px-Monet_-_Valle_Buona%2C_Near_Bordighera%2C_1884.jpg" style="width:512px" /></p>

This is Monet's *Valle Buona Near Bordighera*, 1884 (photo credit: Jerry Ward / Dallas Museum of Art, via [Wikimedia Commons](https://commons.m.wikimedia.org/wiki/File:Monet_-_Valle_Buona,_Near_Bordighera,_1884.jpg), license [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/deed.en)). Actually, the TensorFlow Hub style transfer model predicted very well how Monet might have painted the scene in the content image, but how? Perhaps it's because, the model was trained on arbitrary images which probably included Impressionist paintings, and the Impressionists painted plenty of detailed fields.  So, are we to conclude that content informs style?  Let's flip the script...

In [ ]:
style_path_flip = '../input/gan-getting-started/photo_jpg/00068bc07f.jpg'
content_path_flip = '../input/gan-getting-started/monet_jpg/000c1e3bff.jpg'

content_image_flip = load_img(content_path_flip)
style_image_flip = load_img(style_path_flip)

stylized_image_flip = hub_module(tf.constant(content_image_flip), tf.constant(style_image_flip))[0]
plt.figure(figsize=(15,15))

plt.subplot(1, 3, 1)
imshow(content_image_flip, 'Content Image')

plt.subplot(1, 3, 2)
imshow(stylized_image_flip, 'Result')

plt.subplot(1, 3, 3)
imshow(style_image_flip, 'Style Image')

Wow! Now that we have a modern photographer on the cliffs instead of Monet, we get to see the breaking waves and the clouds in the sky. Those weren't in Monet's painting, or were they?

# Explore Further

The code below selects a random photo and a random Monet and displays results for when Monet is the style and for when his work is the content. To learn more on video -- the BBC television series "Fake or Fortune" Season 1, Episode 1 investigates a painting that might be by Monet (available on at least one major streaming service in the U.S. and possibly available in other countries too).

### Load image filenames

In [ ]:
from os import listdir
from os.path import isfile, join


photo_path = '../input/gan-getting-started/photo_jpg/'
photo_files = [f for f in listdir(photo_path) if isfile(join(photo_path, f))]

monet_path = '../input/gan-getting-started/monet_jpg/'
monet_files = [f for f in listdir(monet_path) if isfile(join(monet_path, f))]
monet_count = len(monet_files)

### Select random photo and random Monet, then stylize and display

In [ ]:
from random import randrange


# Get random photo
content_path1 = photo_path + photo_files[randrange(len(photo_files))]
style_path1 = monet_path + monet_files[randrange(len(monet_files))]

content_image1 = load_img(content_path1)
style_image1 = load_img(style_path1)

content_image2 = style_image1
style_image2 = content_image1

stylized_image1 = hub_module(tf.constant(content_image1), tf.constant(style_image1))[0]
stylized_image2 = hub_module(tf.constant(content_image2), tf.constant(style_image2))[0]

plt.figure(figsize=(20,20))
plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0.25, right=0.8, wspace=0.3)

plt.subplot(2, 3, 1)
imshow(content_image1, 'Content Image')

plt.subplot(2, 3, 2)
imshow(stylized_image1, 'Result')

plt.subplot(2, 3, 3)
imshow(style_image1, 'Style Image')

plt.subplot(2, 3, 4)
imshow(content_image2, 'Content Image')

plt.subplot(2, 3, 5)
imshow(stylized_image2, 'Result')

plt.subplot(2, 3, 6)
imshow(style_image2, 'Style Image')